<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/colorGrading/tests/LPTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Color grading via LPTN

[https://github.com/csjliang/LPTN](https://github.com/csjliang/LPTN)


In [3]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='colorGrading',library='LPTN')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1793  100  1793    0     0  38978      0 --:--:-- --:--:-- --:--:-- 38978
what's the username for nextcloud? colab
what's the password for user colab? ··········
1
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  
content of /etc/fstab: https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ /content/database davfs user,rw,auto 0 0


## 2. Clone the Git repository

In [4]:
import os
root_path = '/content/LPTN'
# clone the repository
if not os.path.exists('LPTN'):
  !git clone https://github.com/csjliang/LPTN {root_path}

Cloning into '/content/LPTN'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 107 (delta 26), reused 67 (delta 6), pack-reused 0
Receiving objects: 100% (107/107), 263.69 KiB | 3.38 MiB/s, done.
Resolving deltas: 100% (26/26), done.


## 3. install dependencies

In [5]:
!pip install -qr {root_path}/requirement.txt

     |████████████████████████████████| 5.8 MB 5.4 MB/s 
     |████████████████████████████████| 190 kB 49.4 MB/s 


## 5. Pretrained model download

In [6]:
# check first if exist, otherwise create path
pretrained_pth = root_path + '/experiments/pretrained_models/net_g_FiveK_numhigh3.pth'

if not os.path.exists(pretrained_pth):
  !gdown --id 11yuFgHqZe9e4OheJ9YHp9lS-4kQC1Fml \
          -O {pretrained_pth}

Downloading...
From: https://drive.google.com/uc?id=11yuFgHqZe9e4OheJ9YHp9lS-4kQC1Fml
To: /content/LPTN/experiments/pretrained_models/net_g_FiveK_numhigh3.pth
100% 2.48M/2.48M [00:00<00:00, 207MB/s]


## 6. install pyYAML and update the YAML file with correct path for pretrained_models

geen idee of we dit nodig hebben, maar heb zelf deze code erin gezet om te checken of het path effectief naar de goeie folder wijst.


In [7]:
!pip install pyYAML

import yaml

# dowload the pre-trained model for colorgrading
 
def read_yaml(LPTN_path):
    """ A function to read YAML file"""
    with open('{}/options/test/LPTN/test_FiveK.yml'.format(LPTN_path)) as f:
        config = yaml.safe_load(f)
 
    return config

read_yaml(root_path)

{'datasets': {'test': {'dataroot_gt': 'datasets/FiveK/FiveK_test_target.lmdb',
   'dataroot_lq': 'datasets/FiveK/FiveK_test_source.lmdb',
   'io_backend': {'type': 'lmdb'},
   'name': 'FiveK_val',
   'type': 'PairedImageDataset'}},
 'manual_seed': 10,
 'model_type': 'LPTNTestModel',
 'name': 'LPTN_FiveK_480p',
 'network_g': {'nrb_high': 3, 'nrb_low': 5, 'num_high': 3, 'type': 'LPTN'},
 'num_gpu': 1,
 'path': {'pretrain_network_g': 'experiments/pretrained_models/net_g_FiveK_numhigh3.pth',
  'strict_load_g': False},
 'val': {'metrics': {'psnr': {'crop_border': 0,
    'test_y_channel': False,
    'type': 'calculate_psnr'}},
  'save_img': True,
  'suffix': None}}

## 7. dataset downloaden

download dataset

In [8]:
import os
import zipfile

# define folders
output_path_dataset = root_path +'/datasets'
output_path_FiveK = output_path_dataset +'/FiveK'
input_path = nextcloud + 'colorGrading/datasets/FiveK_480p.zip'

# make folders
os.makedirs(output_path_dataset, exist_ok=True)
os.makedirs(output_path_FiveK, exist_ok=True)

#!unzip "/content/drive/path/input_file_name.zip" -d "/content/drive/path/output_folder/"
!unzip -q {input_path} -d {output_path_FiveK}

%ls

database/  database_mod.py  LPTN/  __pycache__/  sample_data/


create lmdb

In [7]:
%cd /content/LPTN/
!PYTHONPATH="/content/LPTN/:${PYTHONPATH}" python scripts/data_preparation/create_lmdb.py

/content/LPTN
Reading image path list ...
Create lmdb for datasets/FiveK/FiveK_480p/train/A, save to datasets/FiveK/FiveK_train_source.lmdb...
Totoal images: 2250
Data size per image is:  482882
Write 998: 100% 2250/2250 [01:33<00:00, 24.06chunk/s]

Finish writing lmdb.
Reading image path list ...
Create lmdb for datasets/FiveK/FiveK_480p/train/B, save to datasets/FiveK/FiveK_train_target.lmdb...
Totoal images: 2250
Data size per image is:  678345
Write 999: 100% 2250/2250 [01:46<00:00, 21.15chunk/s]

Finish writing lmdb.
Reading image path list ...
Create lmdb for datasets/FiveK/FiveK_480p/test/A, save to datasets/FiveK/FiveK_test_source.lmdb...
Totoal images: 500
Data size per image is:  308466
Write 5000: 100% 500/500 [00:20<00:00, 23.85chunk/s]

Finish writing lmdb.
Reading image path list ...
Create lmdb for datasets/FiveK/FiveK_480p/test/B, save to datasets/FiveK/FiveK_test_target.lmdb...
Totoal images: 500
Data size per image is:  611200
Write 5000: 100% 500/500 [00:23<00:00, 21

## 8. Evaluation

yml file test

In [8]:
!pip install basicsr
!PYTHONPATH="/content/LPTN/:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test.py -opt options/test/LPTN/test_FiveK.yml

     |████████████████████████████████| 161 kB 4.9 MB/s 
  Created wheel for basicsr: filename=basicsr-1.3.5-py3-none-any.whl size=194484 sha256=8214406e5628981c6876bb3e0a4a51edaaf98ccaafd076b832be7a55ef4d3ad3
  Stored in directory: /root/.cache/pip/wheels/74/1b/d0/8659cf028233dd1e3bf282271009fbf037dfc4ab761f32a032
Successfully built basicsr
Disable distributed.
2022-03-14 08:32:17,032 INFO: 
  name: LPTN_FiveK_480p
  model_type: LPTNTestModel
  num_gpu: 1
  manual_seed: 10
  datasets:[
    test:[
      name: FiveK_val
      type: PairedImageDataset
      dataroot_gt: datasets/FiveK/FiveK_test_target.lmdb
      dataroot_lq: datasets/FiveK/FiveK_test_source.lmdb
      io_backend:[
        type: lmdb
      ]
      phase: test
    ]
  ]
  network_g:[
    type: LPTN
    nrb_low: 5
    nrb_high: 3
    num_high: 3
  ]
  path:[
    pretrain_network_g: experiments/pretrained_models/net_g_FiveK_numhigh3.pth
    strict_load_g: False
    root: /content/LPTN
    results_root: /content/LPTN/results

speed testing

In [9]:
!PYTHONPATH="/content/LPTN/:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml

Disable distributed.
2022-03-14 08:33:50,226 INFO: 
  name: LPTN_test_speed
  model_type: LPTNTestModel
  num_gpu: 1
  manual_seed: 10
  datasets:[
    test:[
      name: FiveK_val
      type: PairedImageDataset
      dataroot_gt: datasets/FiveK/FiveK_test_target.lmdb
      dataroot_lq: datasets/FiveK/FiveK_test_source.lmdb
      io_backend:[
        type: lmdb
      ]
      phase: test
    ]
  ]
  network_g:[
    type: LPTN
    nrb_low: 5
    nrb_high: 3
    num_high: 3
  ]
  path:[
    pretrain_network_g: experiments/pretrained_models/net_g_FiveK_numhigh3.pth
    strict_load_g: False
    root: /content/LPTN
    results_root: /content/LPTN/results/LPTN_test_speed
    log: /content/LPTN/results/LPTN_test_speed
    visualization: /content/LPTN/results/LPTN_test_speed/visualization
  ]
  val:[
    num_img: 10
    times_per_img: 50
    fix_img_size: [3840, 2160]
  ]
  is_train: False
  dist: False
  rank: 0
  world_size: 1

2022-03-14 08:33:50,227 INFO: Dataset PairedImageDataset - FiveK_